# Import libraries

In [9]:
import numpy as np
from warnings import filterwarnings
from cvxpy import Variable, Problem, Maximize, Minimize, log, quad_form
from pulp import LpProblem, LpVariable, LpInteger, LpMaximize, LpMinimize

filterwarnings("ignore")

# 1. Example of convex-concave optimization using `cvxpy`
Before going to the section of `pulp`, we will focus on `cvxpy` and how to establish an optimization-problem with this library!

Also, we have 3 important features to determine : `input_variable`, `objective function` and `constraints`
### 1.1. Define/assign the `input variables`
In this section, we will only consider the optimization problem $(P)$ on $\mathbb{R}^3$, where
$$ x = \left( x_1, x_2, x_3 \right) $$

In [10]:
x1 = Variable()
x2 = Variable()
x3 = Variable()

### 1.2. Define the `objective function`
Here we use 2 type `convex` and `concave` functions

In [11]:
obj_func_1 = Minimize((x1 + x2 - x3)**2)
obj_func_2 = Maximize(log(x1 + x2 + x3 + 1))

### 1.3. Define the contrains

$$ \left\lbrace \begin{array}{clc} x_1 + x_2 &=& 2 \\ x_2 + x_3 & \geq & - 1 \\ x_1 - 2x_3 &=& 0 \end{array} \right. $$

In [12]:
constraints = [x1 + x2 == 2, x2 + x3 >= -1, x1 - 2*x3 == 0]

In [13]:
print("Convex optimization")
prob = Problem(obj_func_1, constraints)
prob.solve()
print(f"x1_opt = {x1.value} \t x2_opt = {x2.value} \t x3_opt = {x3.value}")

Convex optimization
x1_opt = 4.000000000000001 	 x2_opt = -2.000000000000001 	 x3_opt = 2.0000000000000004


In [14]:
print("Concave optimization")
prob = Problem(obj_func_2, constraints)
prob.solve()
print(f"x1_opt = {x1.value} \t x2_opt = {x2.value} \t x3_opt = {x3.value}")

Concave optimization
x1_opt = 5.999999713449373 	 x2_opt = -3.9999997332886355 	 x3_opt = 2.999999856727714


### 1.4. Wrapping up all together with the vector-form

- For example, if $x = \left( x_1,x_2,x_3 \right)$ then the first objective-function becomes

$$ F(x) = (x_1 + x_2 - x_3)^2 = x^T A x $$

where 
$$ A = a^Ta = \left( \begin{array}{ccc} 1 & 1 & -1 \\ 1 & 1 & -1 \\ -1 & -1 & 1 \end{array} \right)$$
and 
$$ a=(1,1,-1) $$

- Likewise, we have 3 distinct contraints here, 
$$ c_1 x = 2 \qquad c_2 x \geq -1 \qquad c_3 x = 0 $$
where
$$ c_1 = (1,1,0) \qquad c_2 = (0,1,1) \qquad c_3 = (1,0,-2) $$

In [15]:
a = np.array([[1,1,-1]])
c1 = np.array([1,1,0])
c2 = np.array([0,1,1])
c3 = np.array([1,0,-2])
x = Variable(3)
vec_cons = [c1@x == 2, c2@x >= -1, c3@x == 0]
Sigma = a.T.dot(a) # equivalent to a.T*a
vec_objf_1 = Minimize(quad_form(x, Sigma)) # equivalent to x.T@(a.T*a)@x
print("Final result")
prob = Problem(vec_objf_1, vec_cons)
prob.solve()
x.value

Final result


array([ 4., -2.,  2.])

- Now, how to do the samething for the other problem

In [16]:
b = np.array([1,1,1])
vec_objf_2 = Maximize(log(b@x + 1))
print("Final result")
prob = Problem(vec_objf_2, vec_cons)
prob.solve()
x.value

Final result


array([ 5.99999971, -3.99999973,  2.99999986])

# 2. Using `pulp`